# Pinecone Demo- Medical Service Decision Prediction Presentation

In [1]:
import pandas as pd
import numpy as np
import pinecone
import openai

import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
messages = [
    {"role": "user", 
     "content": "Which year did Chet Baker first play at Birdland?"
    }
]

# now query WITHOUT context
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',  
    messages=messages,
    temperature=0,
    max_tokens=400,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print(response.choices[0].message.content.strip())

Chet Baker first played at Birdland in 1953.


In [5]:
try:
    # Test API call to list available models
    models = openai.models.list()
    print("API Authentication Successful. Available models:")
    
   
    for model in models.data:
        print(model.id)
except openai.AuthenticationError as e:
    print(f"Authentication failed: {e}")
except openai.RateLimitError as e:
    print(f"Rate limit exceeded: {e}")
except openai.OpenAIError as e:
    print(f"An OpenAI error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

API Authentication Successful. Available models:
tts-1-1106
gpt-3.5-turbo-16k
gpt-4o
tts-1-hd-1106
gpt-3.5-turbo-instruct-0914
davinci-002
tts-1-hd
gpt-3.5-turbo-1106
gpt-4-turbo-2024-04-09
gpt-4-0125-preview
whisper-1
text-embedding-3-large
gpt-3.5-turbo
gpt-3.5-turbo-instruct
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4o-mini-2024-07-18
babbage-002
gpt-4-0613
gpt-4
gpt-4o-mini
gpt-4-1106-preview
dall-e-3
gpt-4-turbo
text-embedding-3-small
text-embedding-ada-002
tts-1
gpt-4o-2024-05-13
dall-e-2


In [6]:
df= pd.read_csv('/Users/josephlim/Downloads/Independent_Medical_Reviews.csv')

In [7]:
df.head(5)

,Reference ID,Report Year,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,Determination,Type,Age Range,Patient Gender,Findings
0,MN16-22639,2016,Infectious,Hepatitis,Pharmacy/Prescription Drugs,Anti-virals,Overturned Decision of Health Plan,Medical Necessity,41-50,Male,Nature of Statutory Criteria/Case Summary: An ...
1,MN16-22638,2016,Mental,Eating Disorder,Mental Health Treatment,Residential Treatment Center - Admission,Upheld Decision of Health Plan,Medical Necessity,21-30,Female,Nature of Statutory Criteria/Case Summary: An...
2,MN16-22637,2016,Autism Spectrum,Autism-PDD-NOS,Autism Related Treatment,Speech Therapy,Upheld Decision of Health Plan,Medical Necessity,0-10,Female,Nature of Statutory Criteria/Case Summary: Th...
3,EI16-22636,2016,Prevention/Good Health,NaN,"Diagnostic Imaging, Screening and Testing",Mammography,Overturned Decision of Health Plan,Experimental/Investigational,65+,Female,Nature of Statutory Criteria/Case Summary: An ...
4,EI06-5319,2006,Cardiac/Circulatory,NaN,Cardio Vascular,NaN,Upheld Decision of Health Plan,Experimental/Investigational,51-64,Male,Physician 1: The patient is a 62-year-old male...


In [8]:
df.shape

(19245, 11)

In [9]:
messages = [
    {"role": "user", 
     "content": "A male in his pre-teen age with Autism Spectrum Disorder has requested Speech Therapy. The parent of an enrollee has requested occupational therapy for one hour per week for treatment of the enrollee’s medical condition. The health plan has made the decision to deny her the treatment. Here are more detailed information about this case: Report Year: 2016, Diagnosis Category:Autism Spectrum, Diagnosis Sub Category: Autism-PDD-NOS, Treatment Category: Autism Related Treatment. Will this decision be upheld?"
    }
]

# now query WITHOUT context
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',  
    messages=messages,
    temperature=0,
    max_tokens=400,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print(response.choices[0].message.content.strip())

It is possible that the decision to deny occupational therapy for the enrollee's medical condition may be upheld, as the treatment requested does not directly align with the diagnosis of Autism Spectrum Disorder. However, it is important to consider the individual needs of the enrollee and the potential benefits that occupational therapy may provide in improving their overall well-being and quality of life. It may be beneficial for the parent to appeal the decision and provide additional information or documentation to support the need for occupational therapy in this case. Ultimately, the final decision will depend on the specific policies and guidelines of the health plan.


In [10]:
from pinecone import Pinecone
api_key= os.getenv('PINECONE_API_KEY')
region= os.getenv('PINECONE_REGION')
pc= Pinecone(api_key= api_key)

In [11]:
from pinecone import ServerlessSpec

cloud= os.getenv('PINECONE_CLOUD')
region= os.getenv('PINECONE_REGION')

spec= ServerlessSpec(cloud= cloud, region= region)

In [12]:
index_name = 'healthcare-predictions'
indexes = pc.list_indexes()

In [13]:
index = pc.Index(index_name)

In [14]:
def get_embeddings(text):
    response = openai.embeddings.create(input=text, model="text-embedding-ada-002")
    return list(response.data[0].embedding)

In [15]:
def predict_decision(text):
    query_vector = get_embeddings(text)

    results = index.query(vector=query_vector, top_k=5, include_metadata=True)

    similar_findings = []
    determinations = []
    similarities = []  # Store similarity scores

    for result in results['matches']:
        metadata = result.get('metadata', {})
        finding = metadata.get('text', "No finding available")
        determination = metadata.get('Determination', "No determination available")
        
        similar_findings.append(finding)
        determinations.append(determination)
        similarities.append(result['score'])  # Get the similarity score

    # Print the vector similarities of top 5 vectors
    print("Top 5 similar vectors and their similarities:")
    for finding, determination, similarity in zip(similar_findings, determinations, similarities):
        print(f"Determination: {determination}, Similarity: {similarity}")

    prompt = (
        "Based on the following similar cases and their determinations, "
        "predict the determination for the new case: "
        + " ".join(f"Finding: {f}, Determination: {d}" for f, d in zip(similar_findings, determinations))
    )
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    for choice in response.choices:
        print(choice)
    
    return response.choices[0].message.content

In [ ]:
while True:
    user_message = input("You: ")
    if user_message.lower() in ['exit', 'quit']:
        break
    response = predict_decision(user_message)
    print("Chatbot:", response)

You: A male in his pre-teen age with Autism Spectrum Disorder has requested Speech Therapy. The parent of an enrollee has requested occupational therapy for one hour per week for treatment of the enrollee’s medical condition. The health plan has made the decision to deny her the treatment. Here are more detailed information about this case: Report Year: 2016, Diagnosis Category:Autism Spectrum, Diagnosis Sub Category: Autism-PDD-NOS, Treatment Category: Autism Related Treatment. Will this decision be upheld?
Top 5 similar vectors and their similarities:
Determination: Overturned Decision of Health Plan, Similarity: 0.934200227
Determination: Overturned Decision of Health Plan, Similarity: 0.933039546
Determination: Overturned Decision of Health Plan, Similarity: 0.9323
Determination: Overturned Decision of Health Plan, Similarity: 0.929806888
Determination: Overturned Decision of Health Plan, Similarity: 0.929252565
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompl

Suggested prompt: A male in his pre-teen age with Autism Spectrum Disorder has requested Speech Therapy. The parent of an enrollee has requested occupational therapy for one hour per week for treatment of the enrollee’s medical condition. The health plan has made the decision to deny her the treatment. Here are more detailed information about this case: Report Year: 2016, Diagnosis Category:Autism Spectrum, Diagnosis Sub Category: Autism-PDD-NOS, Treatment Category: Autism Related Treatment. Will this decision be upheld?